In [193]:
# matplotlib 한글 출력 가능하도록 만들기
from matplotlib import font_manager, rc

font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/gulim.ttc").get_name()
rc('font', family=font_name)

# 데이터 크롤링 모듈
from selenium import webdriver
from bs4 import BeautifulSoup
import re
# 데이터 분석 모듈
import pandas as pd
import numpy as np

In [234]:
# second_crawler로 만든 데이터 로드
df = pd.read_csv('data/statiz_fixed.csv')

# 해당 데이터셋 선수 이름순으로 정렬 및 인덱스 정리
df = df.sort_values(['선수'])
df = df.reset_index(drop=True)
del df['Unnamed: 0']

players = df['선수'].tolist()
births = df['생일'].tolist()

# 연봉 열 추가
df['연봉'] = 0

In [235]:
driver = webdriver.Chrome('webdrivers/chromedriver.exe')
del_list = []

# 선수리스트 순회하며 연봉 데이터 추출
#for i in range(0,1):
for i in range(len(players)):
    if any(df[(df['선수'] == players[i]) & (df['생일'] == births[i])]['연봉'] == 0):
        #print(players[i])
        url = 'http://www.statiz.co.kr/player.php?opt=11&name={}&birth={}'.format(players[i], births[i])
        driver.get(url)
        driver.implicitly_wait(10)

        html = driver.find_element_by_xpath('//*[@class="table table-striped table-condensed"]/tbody').get_attribute(
            "innerHTML")  # 기록 table을 str형태로 저장
        soup = BeautifulSoup(html, 'html.parser')  # str 객체를 BeautifulSoup 객체로 변경

        # players[i] 와 births[i] 일치하는 선수의 연봉 데이터 추출
        temp = [i.text.strip() for i in soup.findAll("tr")]  # tr 태그에서, text만 저장하기
        temp = pd.Series(temp)
        temp = temp[~temp.str.match("[연]")].reset_index(drop=True)
        tmp_list = []

        # 해당 선수 연봉데이터 연도, 연봉으로 분리
        for j in range(len(temp)):
            if j == len(temp)-1:
                tmp_list.append([temp[j][:4], temp[j][4:-5], temp[j][-5:]])
            else:
                if len(temp[j+1].replace('-', '')) < 10:
                    tmp_list.append([temp[j][:4], '2200', temp[j][-5:]])
                else:
                    tmp_list.append([temp[j][:4], temp[j + 1][4:-5], temp[j][-5:]])
                    
        #선수 기록이 1년뿐이면 해당 기록 삭제
        if len(tmp_list) == 1:
            del_list.append(i)
        else:
            # 연봉데이터 리스트 순회하며 df['연봉'] 에 채워넣기
            for t in range(len(tmp_list)):
                print(players[i], tmp_list[t])
                tmp_list[t][1] = tmp_list[t][1].replace(',', '')
                tmp_list[t][1] = tmp_list[t][1].replace('-', '')
                
                if tmp_list[t][1] == '':
                    del_list.append(i)
                    continue
                    
                year = int(tmp_list[t][0])
                salary = int(tmp_list[t][1])
                
                if salary == 900000:
                    salary = tmp_list[t-1][1]
                    tmp_list[t][1] = tmp_list[t-1][1]
                    
                df.loc[(df['선수']==players[i])&(df['연도']==year), '연봉'] = salary
driver.close()

강경학 ['2011', '2,400', '0.005']
강경학 ['2014', '6,500', '0.053']
강경학 ['2015', '6,500-', '0.010']
강경학 ['2016', '6,200-', '0.943']
강경학 ['2017', '5,800', '0.446']
강경학 ['2018', '5,800-', '1.750']
강경학 ['2019', '7,800', '0.168']
강경학 ['2020', '7,800', '0.288']
강귀태 ['2002', '3,000', '0.371']
강귀태 ['2003', '4,000', '0.616']
강귀태 ['2004', '5,500', '0.918']
강귀태 ['2005', '6,300', '2.177']
강귀태 ['2006', '6,300', '1.231']
강귀태 ['2007', '5,000', '0.121']
강귀태 ['2008', '7,000', '0.706']
강귀태 ['2009', '7,000-', '0.656']
강귀태 ['2010', '10,000', '0.021']
강귀태 ['2011', '8,000-', '0.210']
강귀태 ['2012', '8,000-', '0.040']
강동우 ['1998', '3,500', '4.503']
강동우 ['2000', '3,500', '0.018']
강동우 ['2001', '5,500', '1.683']
강동우 ['2002', '10,000', '3.285']
강동우 ['2003', '11,500', '2.466']
강동우 ['2004', '13,500', '1.865']
강동우 ['2005', '13,500', '0.749']
강동우 ['2006', '13,500-', '0.705']
강동우 ['2007', '9,450-', '0.140']
강동우 ['2008', '7,000', '0.391']
강동우 ['2009', '15,000', '4.546']
강동우 ['2010', '15,000', '0.514']
강동우 ['2011', '20,000', 

김대륙 ['2015', '3,200-', '0.371']
김대륙 ['2016', '3,400-', '0.211']
김대륙 ['2017', '3,400-', '0.711']
김대우 ['2009', '2,400-', '0.195']
김대우 ['2010', '2,400-', '0.261']
김대우 ['2012', '2,500', '0.173']
김대우 ['2013', '3,500-', '0.734']
김대우 ['2014', '3,800-', '0.131']
김대우 ['2015', '3,800', '0.982']
김대우 ['2016', '3,000-', '0.009']
김대우 ['2017', '2,900-', '0.240']
김대우 ['2018', '2,900', '0.095']
김대우 ['2020', '2,900', '0.635']
김도환 ['2019', '5,000-', '0.073']
김도환 ['2020', '5,000-', '0.071']
김동엽 ['2016', '4,700', '0.461']
김동엽 ['2017', '11,500-', '1.491']
김동엽 ['2018', '11,500-', '0.123']
김동엽 ['2019', '13,500', '1.020']
김동엽 ['2020', '13,500', '0.467']
김동주 ['1998', '4,000', '3.373']
김동주 ['1999', '6,200', '4.936']
김동주 ['2000', '12,500', '7.278']
김동주 ['2001', '16,500', '2.756']
김동주 ['2002', '22,500', '6.126']
김동주 ['2003', '30,000', '5.610']
김동주 ['2004', '32,000', '5.053']
김동주 ['2005', '42,000', '3.944']
김동주 ['2006', '42,000', '0.636']
김동주 ['2007', '70,000', '7.415']
김동주 ['2008', '70,000', '5.177']
김동주 ['2009', 

김재현 ['2007', '30,000-', '0.000']
김재현 ['2008', '2,400-', '0.072']
김재현 ['2012', '3,200', '0.453']
김재현 ['2013', '4,000', '0.282']
김재현 ['2014', '4,000-', '0.024']
김재현 ['2015', '4,000', '0.193']
김재현 ['2016', '7,000-', '1.368']
김재현 ['2017', '5,100', '0.194']
김재현 ['2018', '5,100', '0.237']
김재현 ['2019', '5,600-', '0.059']
김재현 ['2020', '5,600-', '0.182']
김재현 ['1994', '2200', '5.969']
김재현 ['1995', '2200', '3.206']
김재현 ['1996', '6,000', '3.287']
김재현 ['1998', '10,000', '4.229']
김재현 ['1999', '12,000', '2.927']
김재현 ['2000', '18,000', '4.256']
김재현 ['2001', '18,000', '4.525']
김재현 ['2002', '18,000', '5.031']
김재현 ['2003', '18,000', '1.322']
김재현 ['2004', '23,000', '4.130']
김재현 ['2005', '23,000', '4.681']
김재현 ['2006', '25,000-', '3.362']
김재현 ['2007', '30,000', '0.040']
김재현 ['2008', '50,000', '3.228']
김재현 ['2009', '50,000', '2.512']
김재현 ['2010', '50,000', '1.793']
김재호 ['2004', '2,300', '0.212']
김재호 ['2005', '2,600', '0.155']
김재호 ['2008', '4,400', '1.385']
김재호 ['2009', '5,400', '0.544']
김재호 ['2010', '5,200-

나주환 ['2003', '2,600', '0.377']
나주환 ['2004', '3,300', '0.470']
나주환 ['2005', '5,300', '0.451']
나주환 ['2006', '5,300', '0.289']
나주환 ['2007', '8,700', '0.573']
나주환 ['2008', '12,000', '1.186']
나주환 ['2009', '17,000', '2.694']
나주환 ['2010', '17,000-', '1.646']
나주환 ['2013', '20,000', '0.362']
나주환 ['2014', '20,000-', '0.835']
나주환 ['2015', '20,000', '0.139']
나주환 ['2016', '15,000', '0.320']
나주환 ['2017', '30,000', '1.648']
나주환 ['2018', '30,000-', '1.271']
나주환 ['2019', '12,000', '0.174']
나주환 ['2020', '12,000', '0.201']
나지완 ['2008', '3,800', '1.838']
나지완 ['2009', '10,000', '2.360']
나지완 ['2010', '9,000', '0.540']
나지완 ['2011', '12,500', '2.590']
나지완 ['2012', '15,000', '3.171']
나지완 ['2013', '20,000', '4.230']
나지완 ['2014', '20,000-', '2.376']
나지완 ['2015', '20,000', '0.144']
나지완 ['2016', '60,000', '3.695']
나지완 ['2017', '60,000', '3.399']
나지완 ['2018', '60,000-', '2.458']
나지완 ['2019', '60,000', '0.255']
나지완 ['2020', '60,000', '0.944']
남태혁 ['2016', '3,000-', '0.416']
남태혁 ['2017', '3,300-', '0.110']
남태혁 ['2018

박성한 ['2017', '900,000-', '0.054']
박성한 ['2018', '900,000-', '0.361']
박세혁 ['2012', '2,500', '0.071']
박세혁 ['2013', '3,000', '0.148']
박세혁 ['2016', '6,000', '0.114']
박세혁 ['2017', '10,000', '1.230']
박세혁 ['2018', '10,000', '1.322']
박세혁 ['2019', '23,200', '3.003']
박세혁 ['2020', '23,200', '0.989']
박승욱 ['2012', '2,400-', '0.041']
박승욱 ['2013', '2,700', '0.013']
박승욱 ['2016', '3,500-', '0.511']
박승욱 ['2017', '4,500', '0.238']
박승욱 ['2018', '4,500-', '0.381']
박승욱 ['2019', '8,000-', '0.486']
박승욱 ['2020', '8,000-', '0.241']
박용근 ['2007', '2,800', '0.105']
박용근 ['2008', '3,500', '0.036']
박용근 ['2009', '4,500', '0.218']
박용근 ['2010', '4,000', '0.519']
박용근 ['2014', '4,000-', '0.040']
박용근 ['2015', '4,000-', '0.145']
박용근 ['2016', '4,000-', '0.336']
박용택 ['2002', '5,000', '3.204']
박용택 ['2003', '7,000', '1.750']
박용택 ['2004', '9,000', '2.775']
박용택 ['2005', '15,000', '3.548']
박용택 ['2006', '16,000', '5.065']
박용택 ['2007', '18,000', '3.230']
박용택 ['2008', '15,000', '0.757']
박용택 ['2009', '31,000', '5.794']
박용택 ['2010', '35

...

이원석 ['2005', '3,000-', '0.116']
이원석 ['2006', '5,500', '0.334']
이원석 ['2007', '6,800', '0.709']
이원석 ['2008', '6,800', '0.394']
이원석 ['2009', '10,500', '2.089']
이원석 ['2010', '9,400', '1.517']
이원석 ['2011', '8,400', '0.543']
이원석 ['2012', '10,000', '2.056']
이원석 ['2013', '15,000', '2.810']
이원석 ['2014', '15,000', '0.326']
이원석 ['2016', '30,000', '0.311']
이원석 ['2017', '30,000', '1.116']
이원석 ['2018', '30,000', '3.315']
이원석 ['2019', '30,000', '1.761']
이원석 ['2020', '30,000', '0.604']
이원재 ['2017', '2,700', '0.110']
이원재 ['2018', '2,700-', '0.350']
이원재 ['2019', '7,100-', '0.306']
이원재 ['2020', '7,100-', '0.336']
이인구 ['2003', '1,900-', '0.039']
이인구 ['2004', '2,100', '0.584']
이인구 ['2007', '2,400', '0.247']
이인구 ['2008', '3,700', '1.014']
이인구 ['2009', '6,000-', '1.029']
이인구 ['2010', '4,500', '0.045']
이인구 ['2011', '5,000-', '0.837']
이인구 ['2012', '4,700-', '0.120']
이인구 ['2013', '4,700-', '0.047']
이재원 ['2006', '2,200', '0.246']
이재원 ['2007', '4,000', '0.970']
이재원 ['2008', '5,500', '1.299']
이재원 ['2009', '5,000-'

임훈 ['2004', '2200', '0.039']
임훈 ['2010', '5,000', '0.246']
임훈 ['2011', '8,500', '0.652']
임훈 ['2012', '10,000-', '0.110']
임훈 ['2013', '7,500', '0.402']
임훈 ['2014', '7,500', '1.727']
임훈 ['2015', '13,000-', '0.019']
임훈 ['2016', '10,000-', '0.409']
임훈 ['2017', '10,000-', '0.259']
임훈 ['2018', '10,000-', '0.367']
장민석 ['2001', '2200', '0.080']
장민석 ['2002', '2200', '0.001']
장민석 ['2003', '2200', '0.024']
장민석 ['2008', '2200', '0.107']
장민석 ['2009', '2200', '0.108']
장민석 ['2010', '2200', '2.400']
장민석 ['2011', '2200', '0.078']
장민석 ['2012', '2200', '1.582']
장민석 ['2013', '12,500-', '0.122']
장민석 ['2014', '12,500-', '0.365']
장민석 ['2015', '10,000-', '0.127']
장민석 ['2016', '10,000-', '1.154']
장민석 ['2017', '10,000-', '0.285']
장성우 ['2009', '3,300-', '0.387']
장성우 ['2010', '4,300', '0.351']
장성우 ['2011', '5,500', '0.329']
장성우 ['2014', '5,500', '0.479']
장성우 ['2015', '5,000-', '1.145']
장성우 ['2017', '7,300', '0.005']
장성우 ['2018', '7,300', '0.491']
장성우 ['2019', '13,500', '1.242']
장성우 ['2020', '13,500', '1.497']
장성호

정진호 ['2011', '2,600-', '0.332']
정진호 ['2012', '5,200-', '0.045']
정진호 ['2015', '5,200', '0.255']
정진호 ['2016', '4,700', '0.234']
정진호 ['2017', '8,500', '0.274']
정진호 ['2018', '8,500-', '0.602']
정진호 ['2019', '11,300', '0.498']
정진호 ['2020', '11,300', '0.367']
정현 ['2013', '2,800-', '0.066']
정현 ['2014', '2,800', '0.061']
정현 ['2017', '7,800-', '1.815']
정현 ['2018', '7,800-', '0.584']
정현 ['2019', '3,000-', '0.353']
정현 ['2020', '3,000-', '0.678']
정현석 ['2008', '2,200-', '0.112']
정현석 ['2009', '2,800', '0.288']
정현석 ['2010', '5,000', '0.193']
정현석 ['2013', '8,000-', '2.116']
정현석 ['2014', '9,500', '0.395']
정현석 ['2015', '9,500-', '0.480']
정현석 ['2016', '900,000-', '0.241']
정현석 ['2017', '900,000-', '0.361']
정형식 ['2009', '2,600-', '0.000']
정형식 ['2010', '2,900', '0.028']
정형식 ['2011', '4,000', '0.249']
정형식 ['2012', '6,000', '1.724']
정형식 ['2013', '900,000-', '2.023']
정형식 ['2014', '900,000-', '0.606']
정훈 ['2010', '2,600', '0.282']
정훈 ['2011', '3,000-', '0.399']
정훈 ['2012', '4,200', '0.243']
정훈 ['2013', '8,000', 

최선호 ['2015', '3,500-', '0.580']
최선호 ['2016', '3,500', '0.549']
최선호 ['2019', '3,500', '0.093']
최승준 ['2006', '2,000', '0.000']
최승준 ['2007', '2,400-', '0.020']
최승준 ['2013', '2,700', '0.106']
최승준 ['2014', '2,700-', '0.193']
최승준 ['2015', '4,200', '0.703']
최승준 ['2016', '6,200-', '0.624']
최승준 ['2017', '5,600', '0.143']
최승준 ['2018', '5,600-', '0.059']
최승준 ['2019', '4,000', '0.033']
최승준 ['2020', '4,000', '0.016']
최승환 ['2000', '2,200-', '0.014']
최승환 ['2004', '2,400', '0.047']
최승환 ['2005', '2,800', '0.199']
최승환 ['2006', '3,100', '0.447']
최승환 ['2007', '3,100', '0.010']
최승환 ['2008', '4,200', '0.160']
최승환 ['2009', '5,700', '0.407']
최승환 ['2010', '5,100-', '0.048']
최승환 ['2011', '5,300', '0.182']
최승환 ['2012', '5,000-', '0.484']
최승환 ['2013', '5,000-', '0.140']
최영진 ['2012', '3,300-', '0.171']
최영진 ['2013', '3,500-', '0.126']
최영진 ['2014', '3,000-', '0.111']
최영진 ['2015', '3,000', '0.025']
최영진 ['2017', '3,300', '0.110']
최영진 ['2018', '3,300-', '0.334']
최영진 ['2019', '8,000', '0.157']
최영진 ['2020', '8,000', '0.1

현재윤 ['2002', '2,400-', '0.020']
현재윤 ['2003', '3,000-', '0.283']
현재윤 ['2004', '3,000-', '0.018']
현재윤 ['2007', '6,000', '0.003']
현재윤 ['2008', '7,500', '0.470']
현재윤 ['2009', '10,250-', '0.544']
현재윤 ['2010', '9,000-', '0.073']
현재윤 ['2011', '7,800', '0.080']
현재윤 ['2013', '900,000', '0.316']
현재윤 ['2014', '900,000', '0.284']
홍성갑 ['2014', '3,200-', '0.028']
홍성갑 ['2015', '3,200-', '0.020']
홍성갑 ['2016', '3,600-', '0.287']
홍성갑 ['2017', '3,600-', '0.047']
홍성갑 ['2018', '3,600-', '0.178']
홍성흔 ['1999', '4,000', '1.720']
홍성흔 ['2000', '7,500', '2.963']
홍성흔 ['2001', '12,000', '1.720']
홍성흔 ['2002', '18,000', '4.179']
홍성흔 ['2003', '16,000', '1.652']
홍성흔 ['2004', '28,000', '6.174']
홍성흔 ['2005', '32,000', '2.684']
홍성흔 ['2006', '31,000', '3.655']
홍성흔 ['2007', '31,000', '1.871']
홍성흔 ['2008', '27,900', '2.379']
홍성흔 ['2009', '40,000', '4.190']
홍성흔 ['2010', '40,000', '5.154']
홍성흔 ['2011', '40,000', '2.129']
홍성흔 ['2012', '40,000', '2.713']
홍성흔 ['2013', '40,000', '2.600']
홍성흔 ['2014', '40,000', '3.315']
홍성흔 ['2015

In [239]:
# 해당 기록 당시 선수 연차 계산, 및 df['나이'] column 생성
year = df['연도'].tolist()
birth = df['생일'].tolist()
age = []
for i in range(len(year)):
    age.append(year[i] - int(birth[i][:4]) + 1)
df['나이'] = pd.DataFrame(age)
df

,선수,WAR,G,타석,타수,득점,안타,2타,3타,홈런,...,장타,OPS,wOBA,wRC+,WAR+,연도,포지션,생일,연봉,나이
0,강경학,0.25,52,135,113,21,27,4,0,0,...,0.274,0.615,0.297,77.6,0.25,2019,SS,1992-08-11,7800,28
1,강경학,-0.16,59,93,84,17,18,2,1,0,...,0.262,0.552,0.264,44.1,-0.16,2017,2B,1992-08-11,5800,26
2,강경학,1.38,77,289,245,42,68,11,1,5,...,0.392,0.774,0.353,101.4,1.38,2018,2B,1992-08-11,5800,27
3,강경학,0.11,120,376,311,50,80,7,4,2,...,0.325,0.673,0.304,66.4,0.11,2015,SS,1992-08-11,6500,24
4,강경학,0.21,41,103,86,11,19,2,3,1,...,0.349,0.686,0.316,65.6,0.21,2014,SS,1992-08-11,6500,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,황재균,4.05,124,506,447,78,127,16,3,20,...,0.468,0.825,0.373,130.2,4.05,2019,3B,1987-07-28,120000,33
1483,황재균,2.61,133,504,448,42,122,19,1,4,...,0.346,0.681,0.317,94.7,2.61,2012,3B,1987-07-28,18000,26
1484,황재균,3.76,117,458,398,62,115,18,4,12,...,0.445,0.805,0.368,122.2,3.76,2011,3B,1987-07-28,15000,25
1485,황재균,0.73,94,353,306,41,69,14,3,6,...,0.350,0.653,0.297,70.1,0.73,2010,3B,1987-07-28,10000,24


In [240]:
df.drop(del_list, inplace=True)
df

,선수,WAR,G,타석,타수,득점,안타,2타,3타,홈런,...,장타,OPS,wOBA,wRC+,WAR+,연도,포지션,생일,연봉,나이
0,강경학,0.25,52,135,113,21,27,4,0,0,...,0.274,0.615,0.297,77.6,0.25,2019,SS,1992-08-11,7800,28
1,강경학,-0.16,59,93,84,17,18,2,1,0,...,0.262,0.552,0.264,44.1,-0.16,2017,2B,1992-08-11,5800,26
2,강경학,1.38,77,289,245,42,68,11,1,5,...,0.392,0.774,0.353,101.4,1.38,2018,2B,1992-08-11,5800,27
3,강경학,0.11,120,376,311,50,80,7,4,2,...,0.325,0.673,0.304,66.4,0.11,2015,SS,1992-08-11,6500,24
4,강경학,0.21,41,103,86,11,19,2,3,1,...,0.349,0.686,0.316,65.6,0.21,2014,SS,1992-08-11,6500,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1482,황재균,4.05,124,506,447,78,127,16,3,20,...,0.468,0.825,0.373,130.2,4.05,2019,3B,1987-07-28,120000,33
1483,황재균,2.61,133,504,448,42,122,19,1,4,...,0.346,0.681,0.317,94.7,2.61,2012,3B,1987-07-28,18000,26
1484,황재균,3.76,117,458,398,62,115,18,4,12,...,0.445,0.805,0.368,122.2,3.76,2011,3B,1987-07-28,15000,25
1485,황재균,0.73,94,353,306,41,69,14,3,6,...,0.350,0.653,0.297,70.1,0.73,2010,3B,1987-07-28,10000,24


In [241]:
df.to_csv("data/statiz_final.csv",encoding='utf-8-sig')